<a href="https://colab.research.google.com/github/nidkabo/data_science_projects/blob/main/analyzing_nycflights13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Transformation Project
###Analyzing NYC Flights 2013 Data
**Author**: Nidkamol Boonyanate

In [ ]:
# install and import libraries
install.packages("nycflights13")

library(tidyverse)
library(nycflights13)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.3.0      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Link to data set information: [nycflights13 package](https://www.rdocumentation.org/packages/nycflights13/versions/1.0.1)

This package provides the following data tables.

flights: all flights that departed from NYC in 2013<br>
weather: hourly meterological data for each airport<br>
planes: construction information about each plane<br>
airports: airport names and locations<br>
airlines: translation between two letter carrier codes and names

In [ ]:
# turn all tables into objects so that we can work with them easily
flights <- nycflights13::flights
weather <- nycflights13::weather
planes <- nycflights13::planes
airports <- nycflights13::airports
airlines <- nycflights13::airlines

In [ ]:
# view data structures
glimpse(flights)
cat("\n")
glimpse(weather)
cat("\n")
glimpse(planes)
cat("\n")
glimpse(airports)
cat("\n")
glimpse(airlines)

Rows: 336,776
Columns: 19
$ year           <int> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2…
$ month          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ day            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ dep_time       <int> 517, 533, 542, 544, 554, 554, 555, 557, 557, 558, 558, …
$ sched_dep_time <int> 515, 529, 540, 545, 600, 558, 600, 600, 600, 600, 600, …
$ dep_delay      <dbl> 2, 4, 2, -1, -6, -4, -5, -3, -3, -2, -2, -2, -2, -2, -1…
$ arr_time       <int> 830, 850, 923, 1004, 812, 740, 913, 709, 838, 753, 849,…
$ sched_arr_time <int> 819, 830, 850, 1022, 837, 728, 854, 723, 846, 745, 851,…
$ arr_delay      <dbl> 11, 20, 33, -18, -25, 12, 19, -14, -8, 8, -2, -3, 7, -1…
$ carrier        <chr> "UA", "UA", "AA", "B6", "DL", "UA", "B6", "EV", "B6", "…
$ flight         <int> 1545, 1714, 1141, 725, 461, 1696, 507, 5708, 79, 301, 4…
$ tailnum        <chr> "N14228", "N24211", "N619AA", "N804JB", "N668DN", "N394…
$ origin      

###Question 1: Which airline that travels from JFK had the most flights during the summer (July, August, and September) of 2013?

In [ ]:
flights %>%
    filter(origin == "JFK",               # filter data
            month %in% c(7, 8, 9),
            !is.na(dep_time)) %>%         # filter out all canceled flights
    count(carrier) %>%                    # count flights
    arrange(desc(n)) %>%                  # desc order by the count of flights
    left_join(airlines, by="carrier")     # join to get carrier's names

carrier,n,name
<chr>,<int>,<chr>
B6,11149,JetBlue Airways
DL,5626,Delta Air Lines Inc.
AA,3427,American Airlines Inc.
9E,3370,Endeavor Air Inc.
MQ,1744,Envoy Air
UA,1145,United Air Lines Inc.
VX,906,Virgin America
US,720,US Airways Inc.
EV,344,ExpressJet Airlines Inc.


###Question 2: From question 1, which destinations that people from JFK did travel to the most?

In [ ]:
flights %>%
    filter(origin == "JFK",               # filter data
            month %in% c(7, 8, 9),
            !is.na(dep_time)) %>%         # filter out all canceled flights
    count(dest) %>%                       # count times of travels
    arrange(desc(n)) %>%                  # desc order by the count of travels
    left_join(airports %>%
                select(faa, name),
              by=c("dest"="faa")) %>%     # join to get destination's names
    head()

dest,n,name
<chr>,<int>,<chr>
LAX,2884,Los Angeles Intl
SFO,2109,San Francisco Intl
BOS,1503,General Edward Lawrence Logan Intl
MCO,1390,Orlando Intl
SJU,1247,NA
LAS,1071,Mc Carran Intl


###Question 3: From question 1, What was the total travel distance of each airline that did travel from JFK? How many flights?

In [ ]:
flights %>%
    filter(origin == "JFK",
            month %in% c(7, 8, 9),
            !is.na(dep_time)) %>%
    left_join(airlines, by="carrier") %>%
    group_by(carrier, name) %>%   # group by carrier and name
    summarise(n_flights = n(),    # summarise the number of flights and total distance
              total_dist_mi = sum(distance)) %>%
    arrange(name)                 # order by carrier

`summarise()` has grouped output by 'carrier'. You can override using the
`.groups` argument.


carrier,name,n_flights,total_dist_mi
<chr>,<chr>,<int>,<dbl>
AA,American Airlines Inc.,3427,5706728
DL,Delta Air Lines Inc.,5626,9271491
9E,Endeavor Air Inc.,3370,1822018
MQ,Envoy Air,1744,708351
EV,ExpressJet Airlines Inc.,344,78432
HA,Hawaiian Airlines Inc.,87,433521
B6,JetBlue Airways,11149,12402635
UA,United Air Lines Inc.,1145,2903694
US,US Airways Inc.,720,807028


###Question 4: Which airplane model has the longest total travel distance? How many of them?

In [ ]:
flights %>%
    filter(!is.na(dep_time)) %>%
    inner_join(planes, by="tailnum") %>%
    group_by(model, manufacturer) %>% # group by model and manufacturer
    summarise(n_model = n(),          # summarise the number of model and total distance
              total_dist_mi = sum(distance)) %>%
    arrange(desc(total_dist_mi)) %>%  # order by total distance
    head()

`summarise()` has grouped output by 'model'. You can override using the
`.groups` argument.


model,manufacturer,n_model,total_dist_mi
<chr>,<chr>,<int>,<dbl>
A320-232,AIRBUS,31058,43133359
737-824,BOEING,13809,19819336
757-222,BOEING,9150,19168149
A320-232,AIRBUS INDUSTRIE,14516,18957087
737-832,BOEING,8695,14720241
EMB-145LR,EMBRAER,26603,13632946


###Question 5: Which airlines had the most canceled flights? Make a comparison table between scheduled and canceled flights of each airline.

In [ ]:
cf_data <- flights %>%                               # canceled flights data by carrier
    filter(is.na(dep_time)) %>%                      # filter canceled flights
    group_by(carrier) %>%
    summarise(n_canceled = n())                      # count canceled flights

flights %>% 
    left_join(airlines, by="carrier") %>%            # join related tables
    group_by(carrier, name) %>%                               
    summarise(n_scheduled = n()) %>%                 # count scheduled flights
    left_join(cf_data, by="carrier") %>%             # join canceled flights data
    mutate(cf_prop =                                 # calculate the proportion
      round((n_canceled/n_scheduled), 4)) %>%
    ## cf_prop = canceled flights proportion
    arrange(desc(n_canceled)) %>%                    # order by canceled flights
    head(5)                                          # top 5 most canceled flights airlines

`summarise()` has grouped output by 'carrier'. You can override using the
`.groups` argument.


carrier,name,n_scheduled,n_canceled,cf_prop
<chr>,<chr>,<int>,<int>,<dbl>
EV,ExpressJet Airlines Inc.,54173,2817,0.0520
MQ,Envoy Air,26397,1234,0.0467
9E,Endeavor Air Inc.,18460,1044,0.0566
UA,United Air Lines Inc.,58665,686,0.0117
US,US Airways Inc.,20536,663,0.0323
